In [5]:
import os
import glob
from pathlib import Path
os.chdir("/home/kruu/git_folder/context_learning_aircraft_trajectoires/data")

import pandas as pd
from tqdm.auto import tqdm
from utils import adsb
from traffic.core import Traffic, Flight

import matplotlib.pyplot as plt

****
### Combine raw data
****

In [2]:
path_raw = f"/mnt/beegfs/store/kruu/context_learning/datasets/01_LFPO-LFBO"
path_combined = f"/mnt/beegfs/store/kruu/context_learning/datasets/02_LFPO-LFBO_combined"

adsb.combine_adsb(path_raw = path_raw, path_combined = path_combined)

100%|██████████| 36/36 [01:18<00:00,  2.19s/it]


****
### Processing adsb data
****

#### Take airborne + filter outliers

In [2]:
path_combined = f"/mnt/beegfs/store/kruu/context_learning/datasets/02_LFPO-LFBO_combined"
path_processed = f"/mnt/beegfs/store/kruu/context_learning/datasets/03_LFPO-LFBO_processed"
files = glob.glob(f"{path_combined}/*.parquet")

In [3]:
# Delete trajectories with ovious outlier positions
# 800 Nm seems a good compromise

def delete_outliers(flight, threshold = 800):
    if flight.query(f"cumdist > {threshold}"):
        return None
    else:
        return flight

In [ ]:
# Process each file
for file in files:
    # Info about file being processed
    year, month = file.split(os.sep)[-1].split(".")[0].split("_")
    print(f"Processing {year}/{month}")
    check_file = Path(f"{path_processed}/{year}_{month}.parquet")
    if check_file.is_file() is False:
        # Load monthly file
        t = Traffic.from_file(file)
        # Preprocess
        t = (
            t
            .assign_id()
            .airborne()
            .cumulative_distance()
            .pipe(delete_outliers)
            .eval(max_workers = 30)
        )
        # Save
        if not os.path.exists(f"{path_processed}"):
            os.makedirs(f"{path_processed}")
        t.to_parquet(f"{path_processed}/{year}_{month}.parquet")

#### Merging traffics

In [9]:
path_processed = f"/mnt/beegfs/store/kruu/context_learning/datasets/03_LFPO-LFBO_processed"

files = glob.glob(
    "/mnt/beegfs/store/kruu/context_learning/datasets/03_LFPO-LFBO_processed/*.parquet"
)

path_save = f"/mnt/beegfs/store/kruu/context_learning/datasets/04_LFPO-LFBO_merged"

In [13]:
#merging all traffics

merged_traffic = Traffic(
    pd.concat(
        [
            Traffic.from_file(file).data
            for file in tqdm(files, desc = "Loading files")
        ],
        ignore_index=True,
    )
)

if not os.path.exists(f"{path_save}"):
    os.makedirs(f"{path_save}")

merged_traffic.to_parquet(
    f"{path_save}/all_flights.parquet"
)

Loading files: 100%|██████████| 33/33 [00:36<00:00,  1.11s/it]


In [119]:
# #eurofirs["LFFF"] and eurofirs["LFBB"]

# from cartes.crs import LambertConformal
# from cartes.utils.features import countries, lakes, ocean

# from traffic.data import eurofirs, airports

# fig, ax = plt.subplots(
#     1, 1, dpi=300,
#     subplot_kw=dict(projection=LambertConformal(10, 45)),
# )

# title_style = dict(loc="left", fontsize=12, fontweight=200)

# ax.set_extent((-1, 4, 43 , 49))

# ax.add_feature(countries(scale="50m"))
# ax.add_feature(lakes(scale="50m"))
# ax.add_feature(ocean(scale="50m"))

# ax.spines["geo"].set_visible(False)

# for fir in eurofirs:
#     fir.plot(ax, color="#4c78a833")
#     fir.plot(ax, edgecolor="#4c78a8")

# airports["LFPO"].point.plot(ax, text_kw=dict(s="  Paris", fontsize=8))
# airports["LFBO"].point.plot(ax, text_kw=dict(s="  Toulouse", fontsize=8))

# outliers.plot(ax, color="#f58518", alpha = 0.3)
# ax.set_title("LFPO to LFBO 2022.12", **title_style)

# fig.set_tight_layout(True)

# fig.show()
# fig.savefig("observed_traffic.png", transparent=False)